# Testing VPU & Signal Reconstruction

The aim of this blog post is to test that we can correctly reconstruct a simple signal. And also to better understand the data of each stage.

## Test Binary Vectors
Let's test first by randomly selecting between [0, 0] and [1, 1] vectors.

In [1]:
from src.var_processor.vpu import VPU, VPUBinary
import random
import numpy as np

In [2]:
random.random()

0.46799349226432885

In [3]:
def rand_same():
    a = np.empty([2, 1])
    a.fill(np.random.randint(2))
    return a

In [4]:
def rand_diff():
    a = np.zeros([2, 1])
    index = np.random.randint(2)
    a[index] = 1
    return a

In [5]:
print(rand_same(), rand_same().shape)

[[1.]
 [1.]] (2, 1)


In [6]:
print(rand_diff(), rand_diff().shape)

[[1.]
 [0.]] (2, 1)


In [7]:
vpu = VPU(2)
print(vpu.cu.covariance, vpu.pi.ev)
for _ in range(0, 1000):
    vpu.update_cov(rand_same())
    vpu.pi.iterate(cov=vpu.cu.covariance)
print(vpu.cu.covariance, vpu.pi.ev)

[[0. 0.]
 [0. 0.]] [[0.81341381]
 [0.58168546]]
[[0.24586054 0.24586054]
 [0.24586054 0.24586054]] [[0.70710678]
 [0.70710678]]


In [8]:
print(vpu.pi.eigenvalue, vpu.pi.eigenvector)

[[0.49172109]] [[0.70710678]
 [0.70710678]]


In [9]:
np.sqrt(vpu.pi.eigenvalue)

array([[0.70122827]])

In [10]:
np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))

array([[0.49584326],
       [0.49584326]])

Does the above expression thus give you something to do with the probability for the vector?

In [11]:
np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean

array([[0.96384326],
       [0.96384326]])

We can use this to same with x = 1 or 0? Or x = -1, or 1?

In [12]:
print(
    np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean,
    -1*np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
)

[[0.96384326]
 [0.96384326]] [[-0.02784326]
 [-0.02784326]]


Yes - x = -1 or 1 or random value within this range.

In [26]:
# Adapt to test

vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_same())
    vpu.pi.iterate(cov=vpu.cu.covariance)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance[0])
assert np.allclose(vpu.cu.covariance, vpu.cu.covariance[0])
# Check eigenvector has values of root 2
print(vpu.pi.eigenvector, 1/np.sqrt(2))
assert np.allclose(vpu.pi.eigenvector, 1/np.sqrt(2))
sample_1 = np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
sample_minus1 = -1*np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
assert np.allclose(sample_1, np.array([1, 1]), rtol=0.05, atol=0.05)
assert np.allclose(sample_minus1, np.array([0, 0]), rtol=0.05, atol=0.05)

[[0.24691728 0.24691728]
 [0.24691728 0.24691728]] [0.24691728 0.24691728]
[[0.70710678]
 [0.70710678]] 0.7071067811865475


In [24]:
1/np.sqrt(2)

0.7071067811865475

In [13]:
vpu = VPU(2)
print(vpu.cu.covariance, vpu.pi.ev, sep="\n", end="\n\n")
for _ in range(0, 1000):
    vpu.update_cov(rand_diff())
    vpu.pi.iterate(cov=vpu.cu.covariance)
print(vpu.cu.covariance, vpu.pi.ev, vpu.pi.eigenvalue, sep="\n", end="\n\n")
print(
    vpu.cu.mean,
    np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue)), 
    np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean,
    sep="\n",
    end="\n\n"
)

[[0. 0.]
 [0. 0.]]
[[0.79083108]
 [0.61203448]]

[[ 0.24667776 -0.24667776]
 [-0.24667776  0.24667776]]
[[ 0.70710678]
 [-0.70710678]]
[[0.49335551]]

[[0.486]
 [0.514]]
[[ 0.49666665]
 [-0.49666665]]
[[0.98266665]
 [0.01733335]]



How would we get a sample of [0, 1] if x = 0 then we just have mean. Ah - x needs to be between -1 and 1.

In [14]:
# Forward Pass
a = rand_diff()
print(a, np.dot(vpu.pi.eigenvector.T, a))

[[0.]
 [1.]] [[-0.70710678]]


In [15]:
np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean

array([[0.98266665],
       [0.01733335]])

In [16]:
-1*np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean

array([[-0.01066665],
       [ 1.01066665]])

In [34]:
# Adapt to test

vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_diff())
    vpu.pi.iterate(cov=vpu.cu.covariance)
# Check diagonal values of covariance matrix are the same
# Use https://docs.scipy.org/doc/numpy/reference/generated/numpy.diagonal.html
print(vpu.cu.covariance[0], -1*vpu.cu.covariance[-1])
assert np.allclose(vpu.cu.covariance[0], -1*vpu.cu.covariance[-1])
# Check eigenvector has values of root 2
print(vpu.pi.eigenvector, 1/np.sqrt(2))
assert np.allclose(np.abs(vpu.pi.eigenvector), 1/np.sqrt(2))
# Check different signs
assert np.allclose(vpu.pi.eigenvector[0], -1*vpu.pi.eigenvector[1])
sample_1 = np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
sample_minus1 = -1*np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
print(sample_1, np.flipud(sample_minus1))
assert np.allclose(sample_1, np.flipud(sample_minus1), rtol=0.1, atol=0.1)

[ 0.24607419 -0.24607419] [ 0.24607419 -0.24607419]
[[ 0.70710678]
 [-0.70710678]] 0.7071067811865475
[[ 1.02705866]
 [-0.02705866]] [[0.96505866]
 [0.03494134]]


## Test Reconstruction

So we do a forward pass - get the output r, reconstruct then subtract.

In [35]:
# Setup VPU
vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_same())
    vpu.pi.iterate(cov=vpu.cu.covariance)

In [40]:
# Test forward pass with no processing
ran = rand_same()
r = vpu.forward(ran)
print(ran, r)

[[1.]
 [1.]] [[1.41421356]]


So [0, 0] r = 0 and for [1, 1] r = 1.414.

In [41]:
vpu.cu.mean

array([[0.509],
       [0.509]])

In [68]:
# Test forward pass with mean removal
ran = rand_same()
r = vpu.forward(ran-vpu.cu.mean)
print(ran, r)

[[0.]
 [0.]] [[-0.7198347]]


Now we have [0, 0] = -0.7198, [1, 1] = 0.69437886.

In [52]:
print(vpu.pi.eigenvector, vpu.pi.eigenvalue)
vpu.pi.eigenvector / np.sqrt(vpu.pi.eigenvalue)

[[0.70710678]
 [0.70710678]] [[0.49364388]]


array([[1.00641736],
       [1.00641736]])

In [59]:
# Test forward pass with mean removal and divide by eigenvalue - SAME
ran = rand_same()
r = vpu.forward((ran-vpu.cu.mean)/np.sqrt(vpu.pi.eigenvalue))
print(ran, r)

[[0.]
 [0.]] [[-1.02453288]]


Now we have outputs between -1 and 1.

In [67]:
# Test forward pass with mean removal and divide by eigenvalue - DIFFERENCES
ran = rand_diff()
r = vpu.forward((ran-vpu.cu.mean)/np.sqrt(vpu.pi.eigenvalue))
print(ran, r)

[[0.]
 [1.]] [[-0.01811551]]


Non-feature outputs are then 0.

### Test Signal Reconstruction Over Time

Let's say we have a random 8-bit static signal - [28, 234] - let's experiment with putting this through the system and looking at the output over 255 time steps.

In [115]:
data_in = np.random.randint(255, size=(2, 1)); print(data_in)

[[19]
 [76]]


In [116]:
# Determine signal mean - we'll do a test version
signal_mean = np.random.randint(255, size=(2, 255)).mean(axis=1).reshape(2,1); print(signal_mean)

[[128.99215686]
 [129.31764706]]


In [117]:
# Subtract mean and binary threshold +ve and -ve
zero_mean = data_in - signal_mean; print(zero_mean)

[[-109.99215686]
 [ -53.31764706]]


To cope with the sign we can just - get an array indicating the sign, threshold the absolute, then re-apply the sign.

Use - https://docs.scipy.org/doc/numpy/reference/generated/numpy.sign.html.

But we need to know the range for determining - does this require symmetric ranges either side of the mean? It will be 255-min_mean

In [118]:
signs = np.sign(zero_mean); print(signs)

[[-1.]
 [-1.]]


In [119]:
pbt_range = int(255-signal_mean.min()); print(pbt_range)

126


In [120]:
rand_vals = np.random.randint(pbt_range, size=data_in.shape)
binary_values = np.where(np.abs(zero_mean) > rand_vals, 1, 0)
print(binary_values)

[[1]
 [1]]


In [151]:
# Turn into a short function
def signal_pre_processor(signal, mean):
    """Remove mean and convert to range {-1, 0, 1}"""
    zero_mean = signal - mean
    signs = np.sign(zero_mean)
    rand_vals = np.random.uniform(size=zero_mean.shape)*mean
    binary_values = np.where(np.abs(zero_mean) > rand_vals, 1, 0)
    return binary_values*signs

In [155]:
signal_pre_processor(data_in, signal_mean)

array([[-1.],
       [-0.]])

In [131]:
summed = signal_pre_processor(data_in, signal_mean)
for _ in range(0, int(signal_mean.min())):
    summed += signal_pre_processor(data_in, signal_mean)
print(summed+signal_mean)

[[18.99215686]
 [70.31764706]]


Why are we getting 225 and 80? Something to do with the pbt_range? No - we need to make sure we are summing over the signal mean and then adding the mean.

This can likely be a scalar mean. But if our signals are all different? 

Need to evaluate each element in multiples of the mean value.

We can add this to the sensor routine? DONE

In [135]:
print(255 // signal_mean, 255 % signal_mean, sep="\n")

[[1.]
 [1.]]
[[126.00784314]
 [125.68235294]]


In [136]:
print(3467 // signal_mean, 3467 % signal_mean, sep="\n")

[[26.]
 [26.]]
[[113.20392157]
 [104.74117647]]


In [137]:
summed = signal_pre_processor(data_in, signal_mean)
for _ in range(0, 3467):
    summed += signal_pre_processor(data_in, signal_mean)
print(summed)

[[-3018.]
 [-1495.]]


In [143]:
(summed / (3467 / signal_mean))+signal_mean

array([[16.70535865],
       [73.55477358]])

So we just divide the sum by the (total count / signal mean).

To determine we can generate a random number between 0 and 1 and times by the signal mean.

In [150]:
rand_vals = np.random.uniform(size=zero_mean.shape)*signal_mean
binary_values = np.where(np.abs(zero_mean) > rand_vals, 1, 0)
print(zero_mean, rand_vals, binary_values, sep="\n")

[[-109.99215686]
 [ -53.31764706]]
[[ 66.18549605]
 [125.70555556]]
[[1]
 [0]]


The above is now added to sensor. Anyway back to reconstruction.

# Reconstructing a Static Signal